In [1]:
%%capture
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install --upgrade bitsandbytes
%pip install -U transformers
%pip install --upgrade typing_extensions

In [ ]:
from huggingface_hub import login
import os

hf_token = ""

login(hf_token)

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import json, random

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_dir = "Qwen/Qwen3-8B-Base"
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,   
    device_map="auto",  
    torch_dtype=torch.bfloat16,
    trust_remote_code=True             
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [4]:
train_prompt_style = """Below is a fixed instruction that guides the assistant to write a Korean patent specification in full compliance with Korean Patent Law. 
The assistant must use the provided invention details to generate a structured patent section in Korean.
Think step-by-step before responding
The final response should be written in Korean and follow the 2007 model specification guidelines.
**Respond in Korean.**

### Task Type: {task_type}

### Instruction:
{instruction}

### Context:
{context}

### Input:
{input}

### Output:
{output}"""

# 각 태스크별 instruction 정의
TASK_INSTRUCTIONS = {
    "PATENT_FORM": """당신은 대한민국 특허법에 따라 명세서를 작성하는 특허 어시스턴트입니다. 아래의 구성요소별로 정확하고 구조화된 문서를 생성해야 합니다. 
모든 항목은 특허법 제42조 및 시행규칙 제21조, 개정된 모범 명세서 작성법(2007.07.01. 이후 적용)을 철저히 준수해야 합니다.

1. [발명의 명칭] - 발명의 내용을 간명하게 표현하는 명칭을 작성합니다. 영문명을 {{}} 안에 함께 기재합니다.
2. [기술분야] - 본 발명이 속하는 기술분야를 간결하게 설명합니다.
3. [배경기술] - 종래 기술을 서술하고, 가능하면 문헌 인용을 포함합니다.
4. [해결하려는 과제] - 기존 기술의 문제점과 해결하고자 하는 과제를 기술합니다.
5. [과제의 해결 수단] - 본 발명이 과제를 어떻게 해결하는지를 구체적으로 기술하고, 청구항과 연결합니다.
6. [발명의 효과] - 본 발명이 제공하는 구체적이고 비교 가능한 기술적 효과를 기술합니다.
7. [발명을 실시하기 위한 구체적인 내용] - 실시예, 수단, 수치 등을 포함하여, 평균 기술자가 재현 가능하도록 작성합니다.
8. [도면의 간단한 설명] - 도면이 있을 경우, 각 도면이 무엇을 나타내는지 기술합니다.
9. [특허청구범위] - 독립항 및 종속항을 번호와 함께 명확히 구분하여 작성합니다.""",

    "PATENT_EVALUATION": """제출된 특허 문서 또는 아이디어를 평가하고 개선 방향을 제안하세요. 
다음 기준에 따라 분석하고 점수와 구체적인 개선사항을 제시하세요:
1. [신규성] - 동일한 발명이 이미 국내외에 공개된 경우 특허를 받을 수 없습니다.
2. [진보성] - 해당 기술이 통상의 기술자에게 자명한 경우 특허를 받을 수 없습니다.
3. [산업적 이용 가능성] - 발명이 산업적으로 활용 가능해야 합니다.
4. [기재불비] - 명세서에 발명의 내용을 명확하고 완전하게 기재해야 하며, 통상의 기술자가 이를 재현할 수 있어야 합니다.
각 항목별로 점수(1-10점)와 상세한 평가 의견을 제공하세요.""",

#     "PATENT_RECOMMENDATION": """평가에서 지적된 부분을 반영하여 명세서를 개선하되, 모든 항목(제목 포함)을 생략하지 말고 그대로 유지하세요. 
# 항목 이름, 순서, 구성은 유지하되, 각 항목의 문장이나 내용을 평가에 따라 보완하거나 개선하세요.
# 기존 명세서의 전체 구조를 유지하면서 평가 의견을 반영한 수정된 전체 명세서를 제공하세요.""",

    "PATENT_MODIFICATION": """당신은 특허 명세서 작성 전문가입니다. 
사용자의 요청에 따라 특허 명세서를 전문적이고 정확하게 수정하며, 섹션 간의 일관성을 유지합니다.
수정 시 특허법 및 관련 규정을 준수하고, 기술적 정확성과 법적 유효성을 확보하세요."""
}

In [ ]:
EOS_TOKEN = tokenizer.eos_token

import json
import random
from datasets import Dataset

# 통합 프롬프트 템플릿 (Cell 4에서 복사)
unified_prompt_style = """Below is a fixed instruction that guides the assistant to work as a Korean patent AI assistant.
The assistant must identify the task type and respond accordingly in Korean.
Think step-by-step before responding.
The final response should be written in Korean.
**Respond in Korean.**

### Task Type: {task_type}

### Instruction:
{instruction}

### Context:
{context}

### Input:
{input}

### Output:
{output}"""

def load_jsonl_dataset(path):
    """JSONL 파일을 로드하여 리스트로 반환"""
    with open(path, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    return data

def load_and_process_multi_task_data():
    """3개의 태스크 데이터를 로드하고 통합 (추천 데이터 제외)"""
    
    # 파일명과 태스크 타입 매핑 (추천 데이터 주석처리)
    file_task_mapping = {
        "data/patent_data.jsonl": "PATENT_FORM",
        "data/evaluation_data.jsonl": "PATENT_EVALUATION", 
        # "recomend_data.jsonl": "PATENT_RECOMMENDATION",  # TODO: 추천 데이터 준비되면 주석 해제
        "data/ai_revise_data.jsonl": "PATENT_MODIFICATION"
    }
    
    all_data = []
    
    for filename, task_type in file_task_mapping.items():
        print(f"Loading {filename} for {task_type}...")
        
        try:
            # JSONL 파일 로드
            task_data = load_jsonl_dataset(filename)
            
            # 통합 형태로 변환 (모든 값을 문자열로 강제 변환)
            for item in task_data:
                unified_item = {
                    "task_type": task_type,
                    "instruction": TASK_INSTRUCTIONS[task_type],
                    "context": str(item.get("context", "없음")),  # str() 추가
                    "input": str(item["input"]),                  # str() 추가
                    "output": str(item["output"])                 # str() 추가
                }
                all_data.append(unified_item)
            
            print(f"  Loaded {len(task_data)} samples for {task_type}")
            
        except FileNotFoundError:
            print(f"  Warning: {filename} not found, skipping...")
            continue
    
    # 데이터 셔플
    random.shuffle(all_data)
    
    print(f"\nTotal unified data: {len(all_data)}")
    print("Task distribution:")
    for task_type in set(item["task_type"] for item in all_data):
        count = sum(1 for item in all_data if item["task_type"] == task_type)
        print(f"  {task_type}: {count}")
    
    return all_data

def create_balanced_dataset(all_data, min_samples_per_task=2000):
    """각 태스크별로 균형잡힌 데이터셋 생성"""
    task_data = {}
    
    # 태스크별로 데이터 분리
    for item in all_data:
        task_type = item["task_type"]
        if task_type not in task_data:
            task_data[task_type] = []
        task_data[task_type].append(item)
    
    # 각 태스크별로 샘플링
    balanced_data = []
    for task_type, data in task_data.items():
        if len(data) >= min_samples_per_task:
            # 충분한 데이터가 있으면 랜덤 샘플링
            sampled = random.sample(data, min_samples_per_task)
        else:
            # 부족하면 오버샘플링 (중복 허용)
            sampled = data * (min_samples_per_task // len(data) + 1)
            sampled = sampled[:min_samples_per_task]
        
        balanced_data.extend(sampled)
        print(f"{task_type}: {len(sampled)} samples (original: {len(data)})")
    
    random.shuffle(balanced_data)
    return balanced_data

def formatting_unified_prompts_func(examples):
    """통합 데이터를 프롬프트 형태로 변환"""
    task_types = examples["task_type"]
    instructions = examples["instruction"] 
    contexts = examples["context"]
    inputs = examples["input"]
    outputs = examples["output"]
    
    texts = []
    
    for task_type, instruction, context, inp, out in zip(
        task_types, instructions, contexts, inputs, outputs
    ):
        if not out.endswith(EOS_TOKEN):
            out += EOS_TOKEN
            
        text = unified_prompt_style.format(
            task_type=task_type,
            instruction=instruction,
            context=context,
            input=inp,
            output=out
        )
        texts.append(text)
    
    return {"text": texts}

# 데이터 로드 및 처리
print("🔄 Loading multi-task data...")
all_data = load_and_process_multi_task_data()

print("\n🔄 Creating balanced dataset...")
balanced_data = create_balanced_dataset(all_data, min_samples_per_task=2000)

print(f"\n✅ Final balanced dataset size: {len(balanced_data)}")

# Dataset 객체 생성
dataset = Dataset.from_list(balanced_data)

# 프롬프트 형태로 변환
dataset = dataset.map(
    formatting_unified_prompts_func,
    batched=True,
)

print(f"✅ Dataset after prompt formatting: {len(dataset)}")
print("\n📋 Sample data preview:")
print("-" * 50)
print(dataset["text"][0][:800] + "...")
print("-" * 50)

🔄 Loading multi-task data...
Loading data/patent_data.jsonl for PATENT_FORM...
  Loaded 1000 samples for PATENT_FORM
Loading data/evaluation_data.jsonl for PATENT_EVALUATION...
  Loaded 1997 samples for PATENT_EVALUATION
Loading data/ai_revise_data.jsonl for PATENT_MODIFICATION...
  Loaded 3022 samples for PATENT_MODIFICATION

Total unified data: 6019
Task distribution:
  PATENT_FORM: 1000
  PATENT_EVALUATION: 1997
  PATENT_MODIFICATION: 3022

🔄 Creating balanced dataset...
PATENT_EVALUATION: 2000 samples (original: 1997)
PATENT_MODIFICATION: 2000 samples (original: 3022)
PATENT_FORM: 2000 samples (original: 1000)

✅ Final balanced dataset size: 6000


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

✅ Dataset after prompt formatting: 6000

📋 Sample data preview:
--------------------------------------------------
Below is a fixed instruction that guides the assistant to work as a Korean patent AI assistant.
The assistant must identify the task type and respond accordingly in Korean.
Think step-by-step before responding.
The final response should be written in Korean.
**Respond in Korean.**

### Task Type: PATENT_FORM

### Instruction:
당신은 대한민국 특허법에 따라 명세서를 작성하는 특허 어시스턴트입니다. 아래의 구성요소별로 정확하고 구조화된 문서를 생성해야 합니다. 
모든 항목은 특허법 제42조 및 시행규칙 제21조, 개정된 모범 명세서 작성법(2007.07.01. 이후 적용)을 철저히 준수해야 합니다.

1. [발명의 명칭] - 발명의 내용을 간명하게 표현하는 명칭을 작성합니다. 영문명을 {{}} 안에 함께 기재합니다.
2. [기술분야] - 본 발명이 속하는 기술분야를 간결하게 설명합니다.
3. [배경기술] - 종래 기술을 서술하고, 가능하면 문헌 인용을 포함합니다.
4. [해결하려는 과제] - 기존 기술의 문제점과 해결하고자 하는 과제를 기술합니다.
5. [과제의 해결 수단] - 본 발명이 과제를 어떻게 해결하는지를 구체적으로 기술하고, 청구항과 연결합니다.
6. [발명의 효과] - 본 발명이 제공하는 구체적이고 비교 가능한 기술적 효과를 기술합니다.
7. [...
--------------------------------------------------


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
from peft import LoraConfig, get_peft_model

# LoRA config
peft_config = LoraConfig(
    lora_alpha=16,                           # Scaling factor for LoRA
    lora_dropout=0.05,                       # Add slight dropout for regularization
    r=64,                                    # Rank of the LoRA update matrices
    bias="none",                             # No bias reparameterization
    task_type="CAUSAL_LM",                   # Task type: Causal Language Modeling
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # Target modules for LoRA
)

model = get_peft_model(model, peft_config)

In [ ]:
def filter_too_long(example):
    tokenized = tokenizer(example["text"], truncation=False, return_tensors="pt")
    return tokenized["input_ids"].shape[1] <= 131072

dataset = dataset.filter(filter_too_long)


Filter:   0%|          | 0/6000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (146822 > 131072). Running this sequence through the model will result in indexing errors


In [ ]:
# 1. Dataset 분할
split_dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"✅ Train dataset: {len(train_dataset)}")
print(f"✅ Eval dataset: {len(eval_dataset)}")

# 2. Metrics 함수 (optional)
import math
def compute_metrics(eval_preds):
    loss = eval_preds.loss
    return {
        "eval_loss": loss,
        "perplexity": math.exp(loss) if loss < 100 else float("inf")
    }

# 3. 각 데이터셋에서 태스크 분포 확인
def check_task_distribution(dataset_split, split_name):
    print(f"\n📊 {split_name} Task Distribution:")
    task_counts = {}
    for example in dataset_split:
        # 프롬프트에서 태스크 타입 추출
        text = example["text"]
        # 추천 제외
        for task_type in ["PATENT_FORM", "PATENT_EVALUATION", "PATENT_MODIFICATION"]:
            if f"Task Type: {task_type}" in text:
                task_counts[task_type] = task_counts.get(task_type, 0) + 1
                break
    
    for task, count in task_counts.items():
        percentage = (count / len(dataset_split)) * 100
        print(f"  {task}: {count} ({percentage:.1f}%)")

check_task_distribution(train_dataset, "Train")
check_task_distribution(eval_dataset, "Eval")

✅ Train dataset: 5398
✅ Eval dataset: 600

📊 Train Task Distribution:
  PATENT_EVALUATION: 1789 (33.1%)
  PATENT_FORM: 1803 (33.4%)
  PATENT_MODIFICATION: 1806 (33.5%)

📊 Eval Task Distribution:
  PATENT_FORM: 197 (32.8%)
  PATENT_MODIFICATION: 193 (32.2%)
  PATENT_EVALUATION: 210 (35.0%)


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback


# Training Arguments
training_arguments = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    eval_strategy="steps",   # ✅ 평가 실행 조건
    eval_steps=100,                 # ✅ 50 스텝마다 평가
    num_train_epochs=3,
    logging_steps=50,
    warmup_steps=20,
    logging_strategy="steps",
    learning_rate=1e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=3,      # 최대 3개 체크포인트 유지
)

# Initialize the Trainer
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

Converting train dataset to ChatML:   0%|          | 0/5398 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/5398 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5398 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/5398 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/600 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/600 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/600 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/600 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()
model.config.use_cache = False
trainer.train()

Step,Training Loss,Validation Loss
100,0.602700,0.590532
200,0.587200,0.573379
300,0.538700,0.566305
400,0.606900,0.560636
500,0.574800,0.556949
600,0.560400,0.551803
700,0.549200,0.550315
800,0.532600,0.546494
900,0.545700,0.544862
1000,0.538100,0.541330


TrainOutput(global_step=5900, training_loss=0.5070423585277493, metrics={'train_runtime': 7918.2825, 'train_samples_per_second': 2.045, 'train_steps_per_second': 1.023, 'total_flos': 5.613608276852736e+17, 'train_loss': 0.5070423585277493})

In [ ]:
# 이 부분 이상함
# 학습 완료 후 가장 좋은 모델 저장
trainer.save_model("best_model")  # 디렉토리 이름은 원하는 대로

# tokenizer도 함께 저장 (권장)
tokenizer.save_pretrained("best_model")


('best_model/tokenizer_config.json',
 'best_model/special_tokens_map.json',
 'best_model/chat_template.jinja',
 'best_model/vocab.json',
 'best_model/merges.txt',
 'best_model/added_tokens.json',
 'best_model/tokenizer.json')